In [1]:
#引入模块
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [2]:
#引入数据，此处使用正则表达式对数据进行分割，并忽略报错的数据
train=pd.read_csv('/Users/Bonds/Downloads/NYC_City_Hall_Library_Catalog.csv',delimiter="#|##",error_bad_lines=False,header=0)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
Skipping line 294: Expected 8 fields in line 294, saw 9. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 356: Expected 8 fields in line 356, saw 9. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 730: Expected 8 fields in line 730, saw 9. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 914: Expected 8 fields in line 914, saw 9. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 1108: Expected 8 fields in line 1108, saw 9. Error could possibly be due to quotes being ignored

In [4]:
train

,Corporate Name,Subordinate Unit,Title,Remainder Of Title,Remainder OF Title Page,Date Of Publication,Geographic Name,General Subdivision
0,"""New York City.","Laws, etc.",The charter of the City of New York,with notes thereon,"prepared at the request of the Common Council,...",1851,NaN,
1,"""New York City.","Laws, etc.",The charter of the City of New York,with notes thereon,"prepared at the request of the Common Council,...",1854,NaN,
2,"""New York City.","Laws, etc.",An act to reduce several laws,relative particularly to the City of New York,NaN,1813,NaN,
3,"""New York City.","Laws, etc.",Journal of the convention in relation,begun and held at the City Hall,NaN,1846,NaN,
4,"""New York City.","Laws, etc.",An act to reorganize the local government of t...,passed April 30,NaN,1873,NaN,
5,"""New York State.","Laws, etc.",An act to consolidate into one act and to decl...,being Chapter 410 of the Laws of 1882,NaN,1883,NaN,
6,"""New York City.","Laws, etc.",The Greater New York Charter,also the supplementary acts and constitutional...,by Andrew H. Green.,1897,NaN,
8,"""New York City.","Laws, etc.",The New York City Consolidation Act,together with an appendix of the Royal English...,NaN,1891,NaN,
9,"""New York City.","Laws, etc.",The Greater New York Charter as enacted in 1897,with notes indicating the derivatory statutes ...,by Mark Ash.,1897,New York (N.Y.),Politics and government
10,"""New York City.","Laws, etc.",The Greater New York Charter of 1901,NaN,with notes by J.C. Thomson.,1901,NaN,


In [3]:
train = train.dropna(subset=['Date Of Publication'])
train['Date Of Publication']=train['Date Of Publication'].str.extract(r'(\d{4})')
train['Title']=train['Title'].str.extract(r'((\w*\s*)*)').iloc[:,0]
train['General Subdivision']=train['General Subdivision'].str.extract(r'((\w*\s*)*)')
train['Remainder Of Title'] = train['Remainder Of Title'].str.extract(r'((\w*\s*)*)')

In [5]:
#可以看出数据变整洁了
train

,Corporate Name,Subordinate Unit,Title,Remainder Of Title,Remainder OF Title Page,Date Of Publication,Geographic Name,General Subdivision
0,"""New York City.","Laws, etc.",The charter of the City of New York,with notes thereon,"prepared at the request of the Common Council,...",1851,NaN,
1,"""New York City.","Laws, etc.",The charter of the City of New York,with notes thereon,"prepared at the request of the Common Council,...",1854,NaN,
2,"""New York City.","Laws, etc.",An act to reduce several laws,relative particularly to the City of New York,NaN,1813,NaN,
3,"""New York City.","Laws, etc.",Journal of the convention in relation,begun and held at the City Hall,NaN,1846,NaN,
4,"""New York City.","Laws, etc.",An act to reorganize the local government of t...,passed April 30,NaN,1873,NaN,
5,"""New York State.","Laws, etc.",An act to consolidate into one act and to decl...,being Chapter 410 of the Laws of 1882,NaN,1883,NaN,
6,"""New York City.","Laws, etc.",The Greater New York Charter,also the supplementary acts and constitutional...,by Andrew H. Green.,1897,NaN,
8,"""New York City.","Laws, etc.",The New York City Consolidation Act,together with an appendix of the Royal English...,NaN,1891,NaN,
9,"""New York City.","Laws, etc.",The Greater New York Charter as enacted in 1897,with notes indicating the derivatory statutes ...,by Mark Ash.,1897,New York (N.Y.),Politics and government
10,"""New York City.","Laws, etc.",The Greater New York Charter of 1901,NaN,with notes by J.C. Thomson.,1901,NaN,


In [7]:
#此处使用Remainder OF Title Page这一列进行模型训练，因为此列包含描述信息最多
data = train[['Date Of Publication',"Remainder OF Title Page"]].dropna().drop_duplicates()
data

,Date Of Publication,Remainder OF Title Page
0,1851,"prepared at the request of the Common Council,..."
1,1854,"prepared at the request of the Common Council,..."
6,1897,by Andrew H. Green.
9,1897,by Mark Ash.
10,1901,with notes by J.C. Thomson.
11,1905,arranged and annotated by R.C. Cumming.
12,1901,by Mark Ash.
13,1902,by Mark and William Ash.
14,1906,by Mark Ash and William Ash.
15,1912,by Mark and William Ash.


In [8]:
#将数据集分为训练集和测试集
sp = 8000
X = data['Remainder OF Title Page'].tolist()[:sp]
y = data['Date Of Publication'].tolist()[:sp]
X_test = data['Remainder OF Title Page'].tolist()[sp:]
y_test = data['Date Of Publication'].tolist()[sp:]

In [10]:
#将文字向量化
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X)
X_train_counts.shape

(8000, 8346)

In [11]:
#进行tfidf处理
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(8000, 8346)

In [12]:
#此处利用朴素贝叶斯的MultinomialNB作为模型
clf = MultinomialNB().fit(X_train_tfidf, y)

In [13]:
#建造流水线
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [14]:
#进行训练
text_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [15]:
#evaluate模型
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.027827648114901255

In [ ]:
#answer:
#1.模型使用了朴素贝叶斯的MultinomialNB，精度如上
#2.thinking在comment中
#3.朴素贝叶斯广泛用于文字分类中，且该数据集数据量较小，且缺失值多，朴素贝叶斯对此不敏感
#4.为了提高模型的精度，下一步可以先将数据集分类，例如，将数据分为1970年前和1970年后的两类，再对每一块数据进行模型的训练
#4.总结：该数据集预测年份，分类的类别过多，导致精度大幅下降